# Setup/Test Environment:

In [1]:
from web_footprinting_utils import setup_ipython
import os

get_ipython, INTERACTIVE = setup_ipython()

In [2]:
import sys
sys.executable

'/miniconda/bin/python'

In [3]:
%env TARGET=10.10.129.232

env: TARGET=10.10.129.232


In [4]:
!echo Make sure we are connected to VPN:
result = !ip a | grep tun0

if not len(result)>0:
    print("Not connected!")
    input()
else:
    print("Connected!")
    print(result)

Make sure we are connected to VPN:
Not connected!


# Initial (Unreliable) RustScan:

In [5]:
import os
pwd = os.getcwd()
rustscan = "docker run -v '%s':/workspace  --volume /etc/passwd:/etc/passwd:ro --volume /etc/group:/etc/group:ro --user $(id -u) --rm my-other-github-account/rustscan" % pwd

In [6]:
!{rustscan} -t 4000 -r 1-65535 -a {os.environ["TARGET"]} -- -Pn -A -sV

zsh:1: command not found: docker


# Initial Nmap:

In [7]:
!echo $TARGET
!mkdir -p scans/quick scans/tcp scans/udp scans/scripts scans/http scans/no_ping

10.10.129.232


In [8]:
!ping -c 5 $TARGET

PING 10.10.129.232 (10.10.129.232) 56(84) bytes of data.

--- 10.10.129.232 ping statistics ---
5 packets transmitted, 0 received, 100% packet loss, time 4051ms



# Quick Scan:

In [9]:
!nmap -vv -oA scans/quick/quick_scan $TARGET

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-07 02:43 UTC
Initiating ARP Ping Scan at 02:43
Scanning 10.10.129.232 [1 port]
Completed ARP Ping Scan at 02:43, 0.05s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 02:43
Completed Parallel DNS resolution of 1 host. at 02:43, 0.00s elapsed
Initiating SYN Stealth Scan at 02:43
Scanning ip-10-10-129-232.eu-west-1.compute.internal (10.10.129.232) [1000 ports]
Discovered open port 3389/tcp on 10.10.129.232
Discovered open port 80/tcp on 10.10.129.232
Completed SYN Stealth Scan at 02:44, 4.56s elapsed (1000 total ports)
Nmap scan report for ip-10-10-129-232.eu-west-1.compute.internal (10.10.129.232)
Host is up, received arp-response (0.00037s latency).
Scanned at 2021-10-07 02:43:57 UTC for 5s
Not shown: 998 filtered ports
Reason: 998 no-responses
PORT     STATE SERVICE       REASON
80/tcp   open  http          syn-ack ttl 128
3389/tcp open  ms-wbt-server syn-ack ttl 128
MAC Address: 02:33:AC:39:48:09 (Unknown)

R

# All Ports:

In [10]:
!nmap -p- -sV -vv -oA  scans/tcp/tcp_scan $TARGET

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-07 02:44 UTC
NSE: Loaded 45 scripts for scanning.
Initiating ARP Ping Scan at 02:44
Scanning 10.10.129.232 [1 port]
Completed ARP Ping Scan at 02:44, 0.06s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 02:44
Completed Parallel DNS resolution of 1 host. at 02:44, 0.00s elapsed
Initiating SYN Stealth Scan at 02:44
Scanning ip-10-10-129-232.eu-west-1.compute.internal (10.10.129.232) [65535 ports]
Discovered open port 80/tcp on 10.10.129.232
Discovered open port 3389/tcp on 10.10.129.232
SYN Stealth Scan Timing: About 19.95% done; ETC: 02:46 (0:02:04 remaining)
SYN Stealth Scan Timing: About 48.31% done; ETC: 02:46 (0:01:05 remaining)
Discovered open port 5985/tcp on 10.10.129.232
Completed SYN Stealth Scan at 02:45, 103.97s elapsed (65535 total ports)
Initiating Service scan at 02:45
Scanning 3 services on ip-10-10-129-232.eu-west-1.compute.internal (10.10.129.232)
Completed Service scan at 02:45, 6.12s elapsed 

In [11]:
import xmltodict

with open('scans/tcp/tcp_scan.xml','r') as f:
    xml_dict = xmltodict.parse(f.read())

In [12]:
PING=True
try:
    port_list = [port['@portid'] for port in xml_dict['nmaprun']['host']['ports']['port']]
    %env PORTS={",".join(port_list)}
except:
    print("Host is down")
    PING=False

env: PORTS=80,3389,5985


# No Ping:

In [13]:
!nmap -sV -Pn -vv -oA scans/no_ping/no_ping_scan $TARGET

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-07 02:45 UTC


Host discovery disabled (-Pn). All addresses will be marked 'up' and scan times will be slower.


NSE: Loaded 45 scripts for scanning.
Initiating ARP Ping Scan at 02:45
Scanning 10.10.129.232 [1 port]
Completed ARP Ping Scan at 02:45, 0.05s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 02:45
Completed Parallel DNS resolution of 1 host. at 02:45, 0.00s elapsed
Initiating SYN Stealth Scan at 02:45
Scanning ip-10-10-129-232.eu-west-1.compute.internal (10.10.129.232) [1000 ports]
Discovered open port 80/tcp on 10.10.129.232
Discovered open port 3389/tcp on 10.10.129.232
Completed SYN Stealth Scan at 02:45, 4.87s elapsed (1000 total ports)
Initiating Service scan at 02:45
Scanning 2 services on ip-10-10-129-232.eu-west-1.compute.internal (10.10.129.232)
Completed Service scan at 02:46, 6.01s elapsed (2 services on 1 host)
NSE: Script scanning 10.10.129.232.
NSE: Starting runlevel 1 (of 2) scan.
Initiating NSE at 02:46
Completed NSE at 02:46, 0.01s elapsed
NSE: Starting runlevel 2 (of 2) scan.
Initiating NSE at 02:46
Completed NSE at 02:46, 0.00s elapsed
Nmap s

In [14]:
if not PING:
    !nmap -sV -Pn -p- -vv -oA  scans/tcp/tcp_scan $TARGET

# Run Default Scripts:

In [15]:
if PING:
    !nmap -sC -p $PORTS -vv -oA scans/scripts/script_scan $TARGET

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-07 02:46 UTC
NSE: Loaded 123 scripts for scanning.
NSE: Script Pre-scanning.
NSE: Starting runlevel 1 (of 2) scan.
Initiating NSE at 02:46
Completed NSE at 02:46, 0.00s elapsed
NSE: Starting runlevel 2 (of 2) scan.
Initiating NSE at 02:46
Completed NSE at 02:46, 0.00s elapsed
Initiating ARP Ping Scan at 02:46
Scanning 10.10.129.232 [1 port]
Completed ARP Ping Scan at 02:46, 0.04s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 02:46
Completed Parallel DNS resolution of 1 host. at 02:46, 0.00s elapsed
Initiating SYN Stealth Scan at 02:46
Scanning ip-10-10-129-232.eu-west-1.compute.internal (10.10.129.232) [3 ports]
Discovered open port 80/tcp on 10.10.129.232
Discovered open port 3389/tcp on 10.10.129.232
Discovered open port 5985/tcp on 10.10.129.232
Completed SYN Stealth Scan at 02:46, 0.03s elapsed (3 total ports)
NSE: Script scanning 10.10.129.232.
NSE: Starting runlevel 1 (of 2) scan.
Initiating NSE at 02:4

In [ ]:
if PING:
    !nmap -A -T5 -vv --script=default,safe,discovery $TARGET
else:
    !nmap -A -T5 -Pn -vv --script=default,safe,discovery $TARGET

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-07 02:46 UTC
NSE: Loaded 419 scripts for scanning.
NSE: Script Pre-scanning.
NSE: Starting runlevel 1 (of 3) scan.
Initiating NSE at 02:46
NSE: [targets-ipv6-map4to6] This script is IPv6 only.
NSE: [broadcast-ataoe-discover] No interface supplied, use -e
NSE: [broadcast-sonicwall-discover] No network interface was supplied, aborting.
NSE: [url-snarf] no network interface was supplied, aborting ...
NSE: [shodan-api] Error: Please specify your ShodanAPI key with the shodan-api.apikey argument
NSE: [targets-xml] Need to supply a file name with the targets-xml.iX argument
NSE: [targets-ipv6-wordlist] Need to be executed for IPv6.
NSE: [mtrace] A source IP must be provided through fromip argument.
NSE Timing: About 95.88% done; ETC: 02:46 (0:00:01 remaining)
Completed NSE at 02:46, 40.08s elapsed
NSE: Starting runlevel 2 (of 3) scan.
Initiating NSE at 02:46
Completed NSE at 02:46, 0.00s elapsed
NSE: Starting runlevel 3 (of 3) scan.
Initiati

In [ ]:
if PING:
    !nmap -A -T3 -vv --script=default,safe,discovery $TARGET
else:
    !nmap -A -T3 -Pn -vv --script=default,safe,discovery $TARGET

In [ ]:
from getpass import getpass
import os

if PING:
    !echo { getpass() } | sudo -S nmap -A -p- -T5 -vv --script=default,safe,discovery {os.environ['TARGET']}
else:
    !echo { getpass() } | sudo -S nmap -A -p- -Pn -T5 -vv --script=default,safe,discovery {os.environ['TARGET']}

In [ ]:
# UDP Scan, this takes forever...

# from getpass import getpass

# !echo { getpass() } > tmp.txt
# !cat tmp.txt | sudo -S nmap -sU --open -oA scans/udp/udp_scan $TARGET
# !cat tmp.txt | sudo -S nmap -sU --open -p- -oA scans/udp/udp_scan $TARGET
# !rm -rf tmp.txt